In [1]:
import os
import glob

import cv2
import cvlib as cv
import numpy as np
import pandas as pd
from random import shuffle
from tqdm.notebook import tqdm
from keras.preprocessing.image import ImageDataGenerator


CROP_SIZE = (48,48)
import asyncio
import telegram
import nest_asyncio 
nest_asyncio.apply()
bot = telegram.Bot(token="6441009419:AAGeU_ZUW6pxvqAOwIcIA8-PH8RLRPgb9rQ")
chat_id = "5862996909"

In [2]:
class DataHandle:
    def __init__(self):
        """
        DataHandle class for handling image data processing and saving.

        This class provides methods to detect faces in images, crop and resize them,
        and save the processed images along with corresponding emotion labels.

        """
        self.db = []  # A list to store processed image paths and labels
        self.img = None
        self.X_position = (0, 0)
        self.Y_position = (0, 0)
        self.curr_emotion = ''
        self.new_file_path = ""

    def _save_crop_img(self):
        """
        Crop and save the detected face region from the image.

        Returns:
            bool: True if cropping and saving are successful, False otherwise.

        """
        try:
            img = self.img.copy()
            roi = img[
                self.Y_position[0]:self.Y_position[1],
                self.X_position[0]:self.X_position[1],
            ]
            img = cv2.resize(roi, CROP_SIZE, interpolation=cv2.INTER_CUBIC)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            self.img = img
            return True
        except:
            return False

    def _detect_face(self, img_path):
        """
        Detect a face in the given image.

        Args:
            img_path (str): Path to the image file.

        Returns:
            bool: True if a face is detected, False otherwise.

        """
        try:
            self.img = cv2.imread(img_path)
            faces, _ = cv.detect_face(self.img, enable_gpu=False)
            self.X_position = faces[0][0], faces[0][2]
            self.Y_position = faces[0][1], faces[0][3]
            return True
        except:
            return False

    def _random_name(self):
        """
        Generate a random alphanumeric string as a name.

        Returns:
            str: A randomly generated name.

        """
        rand_int = list(map(str, np.random.randint(low=0, high=9, size=7)))
        lst = list(map(chr, np.random.randint(low=97, high=122, size=35))) + rand_int
        shuffle(lst)
        return "".join(lst)

    def work(self, img_path, emo):
        """
        Process the image and save it with the provided emotion label.

        Args:
            img_path (str): Path to the image file.
            emo (str): Emotion label for the image.

        Returns:
            bool: True if processing and saving are successful, False otherwise.

        """
        self.curr_emotion = emo
        if self._detect_face(img_path) and self._save_crop_img():
            if self.img.shape == CROP_SIZE:
                self.new_file_path = f"./dataset/{self.curr_emotion}_{self._random_name()}.jpg"
                self.db.append({"path": self.new_file_path, "label": self.curr_emotion})
                cv2.imwrite(self.new_file_path, self.img)
                return True
            else:
                return False

In [3]:
# Initialize a DataHandle instance
dbHandle = DataHandle()

# Get the list of subdirectories in the "./pre_dataset/" directory
path_dir_lst = os.listdir("./pre_dataset/")
print(path_dir_lst)

# Create a list of folder paths for each emotion category
folder_lst = [
    f'./pre_dataset/{e}/'
    for e in path_dir_lst
]
print(folder_lst)

# Process images in each folder and save to the database
for emo, folder in zip(path_dir_lst, folder_lst):
    try:
        # Get a list of image file paths in the current folder
        img_list = glob.glob(os.path.join(folder) + "*.jpg")
        img_list = list(map(lambda x: x.replace("\\", '/'), img_list))
        
        # Process each image and save to the database
        for img_path in img_list:
            dbHandle.work(img_path, emo)
    except:
        # Handle errors by sending a message using the Telegram bot
        asyncio.run(bot.send_message(chat_id=chat_id, text="An error occurred"))
        
# Save the database to a CSV file
pd.DataFrame(dbHandle.db).to_csv("dataset.csv", index=False)

# Send a completion message to Telegram
asyncio.run(bot.send_message(chat_id=chat_id, text="Jupyter Notebook execution completed"))

['angry', 'fear', 'happy', 'neutral', 'sad']
['./pre_dataset/angry/', './pre_dataset/fear/', './pre_dataset/happy/', './pre_dataset/neutral/', './pre_dataset/sad/']


Message(channel_chat_created=False, chat=Chat(first_name='SungWook', id=5862996909, last_name='Jung', type=<ChatType.PRIVATE>), date=datetime.datetime(2023, 8, 17, 21, 30, 25, tzinfo=<UTC>), delete_chat_photo=False, from_user=User(first_name='jupyterbot', id=6441009419, is_bot=True, username='oceanstarbot'), group_chat_created=False, message_id=33, supergroup_chat_created=False, text='주피터 완료')